In [1]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

from collections import defaultdict

In [2]:
# Define the path
#path = os.path.join("GitHub", "DataProcessing")

# Print the path
#print(path)

In [3]:
#pwd

In [4]:
#td = pd.read_json("..\DataProcessing\TestData\message.json", orient='index')

In [5]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r", encoding="utf-8") as file:
    postData = json.load(file)
    
with open("../DataProcessing/TestData/runeSheet.json", "r", encoding="utf-8") as file:
    runeSheet = json.load(file)

In [6]:
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']
framesData = data['info']['frames']
participantFrames = [frame['participantFrames'] for frame in framesData]


postParticipantData = postData['info']['participants']
postChallengeData = [participants['challenges'] for participants in postParticipantData]

In [7]:
# Create dictionaries to store the counts and most mentioned names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)
mostMentionedNamesDict = {}


In [8]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

    return championCountsDict, namesDict

# Calculate champion counts
championCountsDict, namesDict = calculateChampionCounts(data)

# Process the counts and choose the most mentioned champion name for each ID
mostMentionedNamesDict = {}
for participantId, championCounts in championCountsDict.items():
    mostMentionedChampion = max(championCounts, key=championCounts.get)
    mostMentionedNamesDict[participantId] = mostMentionedChampion
    count = championCounts[mostMentionedChampion]
    names = namesDict[participantId]

    # print(f"Participant ID: {participantId}, Most Mentioned Champion: {mostMentionedChampion} (Count: {count})")
    # print(f"Names associated with ID {participantId}: {', '.join(names)}\n")

# Access the most mentioned champion name for a specific participant ID
# Example: print(mostMentionedNamesDict[1])
mostMentionedNamesDict


{6: 'Shen',
 1: 'Jax',
 7: 'Khazix',
 2: 'MasterYi',
 8: 'Sylas',
 5: 'Leona',
 4: 'Varus',
 10: 'Nautilus',
 9: 'Ezreal',
 3: 'Veigar'}

In [9]:
participantData = []

participantChampionMapping = {}

def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

participantData[0]


[1,
 '9nFTht795izeoOQoIQMOHtY01trPiRW8SC7kUFIqsDJtTEeNUbpxbRriCUtJ3sApIbpwIBNV44mFjg',
 'Jax']

In [10]:
participantData

[[1,
  '9nFTht795izeoOQoIQMOHtY01trPiRW8SC7kUFIqsDJtTEeNUbpxbRriCUtJ3sApIbpwIBNV44mFjg',
  'Jax'],
 [2,
  '7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg',
  'MasterYi'],
 [3,
  'OosGFQGfn836eMPWWHKI4078RDJNVjtEN5rDdfGWkVKE9WQt57XOtBX7dJbKA2gCjvT2B2oKSOPvQA',
  'Veigar'],
 [4,
  'GDyaku8KMFbw6-NeobFdpBjk0cH7HeEwcO1O6lJDWx5DlKXBJACwQboLx_8YpwIKIjQajzWtoYjFlQ',
  'Varus'],
 [5,
  'YXHmDXvWI5gKk7U3akJzmYR7-SUqINElADzyjLGaKOQ3uo114kFt0bgb9jfrbFyGv3wJY-fss-iGCA',
  'Leona'],
 [6,
  'a8PtUmwWlbJrEzU-nfibnfFXKmsStyk87eyureQ-juwhCH8cT9V7PIro5PogLxdME0DzYgHuuyl-Fg',
  'Shen'],
 [7,
  '735uoGC0wWXxRrYW7td31v_tqqAJkVJ3WIwoDAMnOCVvVL6gTw84KV5LFOQpwFkBdPGOQATvkx6ZHw',
  'Khazix'],
 [8,
  'eVreMXaliubuCZDOSDs-HnMy3YSPNa3BoCKTUDDQBx3uotw36LQHmhYoJNWBxy5v0U5Wt_jV0yR80A',
  'Sylas'],
 [9,
  'esX_22maBbJu3Sf7pHod9r65uxnslhai-EnVbgIGjdC4XhW3_5Pocq561XKKt80bZlP5CPvJ3kCoow',
  'Ezreal'],
 [10,
  'pnMNfdDjM97Gr0Qh67jTzEEfLefEI8qjsBxmojuIqGp7cpQPWeBBZ6jfv_UUeV3DqvUg06Rk3lphTg'

In [11]:
# Function to find participant data in kill events
def findParticipantDataInKillEvents(data):
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    
                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

# Example usage
findParticipantDataInKillEvents(data)

# Process the counts and choose the most mentioned champion name for each ID
for participantId, championCounts in championCountsDict.items():
    mostMentionedChampion = max(championCounts, key=championCounts.get)
    count = championCounts[mostMentionedChampion]
    names = namesDict[participantId]

    # print(f"Participant ID: {participantId}, Most Mentioned Champion: {mostMentionedChampion} (Count: {count})")
    # print(f"Names associated with ID {participantId}: {', '.join(names)}\n")


In [12]:
# Get every timestamp within a game paired with corresponding minute
def getTimestamps():
    for counter, frame in enumerate(data['info']['frames']):
        timestamp = frame['timestamp']
        print(f"{counter + 1} Timestamp = {timestamp}")

#getTimestamps()

In [13]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData

In [14]:
def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key}:")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

In [15]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = "2"
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120029
championStats:
    armor: 36
    attackDamage: 71
    attackSpeed: 114
    health: 417
    healthMax: 761
    healthRegen: 23
    magicResist: 33
    movementSpeed: 355
    power: 236
    powerMax: 281
    powerRegen: 34
  currentGold: 161
damageStats:
    totalDamageDone: 3255
    totalDamageDoneToChampions: 145
    totalDamageTaken: 755
  jungleMinionsKilled: 4
  level: 2
  participantId: 2
position:
    x: 8578
    y: 2559
  totalGold: 611
  xp: 325

---



In [16]:
def getChampionKilled(data, startTimestamp, endTimestamp):
    for frame in data['info']['frames']:
        timestamp = frame['timestamp']
        
        if startTimestamp <= timestamp <= endTimestamp:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killTimestamp = event['timestamp']
                    print(f"\nTimestamp of CHAMPION_KILL event: {killTimestamp}")

                    # Get the champion name of the killed participant using victimId
                    victimId = event.get('victimId')
                    victimData = next((p for p in participantData if p[0] == victimId), None)
                    if victimData:
                        victimChampionName = victimData[2]
                        print(f"Champion killed: {victimChampionName} (ID: {victimId})")

                        print("Victim Damage Received:")
                        processedNames = set()
                        for damageInfo in event.get('victimDamageReceived', []):
                            name = damageInfo['name']
                            if name in [p[2] for p in participantData] and name not in processedNames:
                                print(f"  Name: {name}")
                                processedNames.add(name)
                    else:
                        print(f"No participant data found for victim ID: {victimId}\n")

# Example usage: Specify the start and end timestamps
startTimestamp = 120000  # Replace with your desired start timestamp
endTimestamp = 500000    # Replace with your desired end timestamp

# Call the function with the specified time frame
getChampionKilled(data, startTimestamp, endTimestamp)



Timestamp of CHAMPION_KILL event: 175426
Champion killed: Jax (ID: 1)
Victim Damage Received:
  Name: Shen

Timestamp of CHAMPION_KILL event: 269056
Champion killed: MasterYi (ID: 2)
Victim Damage Received:
  Name: Khazix
  Name: Shen

Timestamp of CHAMPION_KILL event: 272085
Champion killed: Shen (ID: 6)
Victim Damage Received:
  Name: MasterYi
  Name: Jax

Timestamp of CHAMPION_KILL event: 283885
Champion killed: Khazix (ID: 7)
Victim Damage Received:
  Name: MasterYi
  Name: Jax

Timestamp of CHAMPION_KILL event: 384416
Champion killed: MasterYi (ID: 2)
Victim Damage Received:
  Name: Khazix

Timestamp of CHAMPION_KILL event: 384449
Champion killed: Khazix (ID: 7)
Victim Damage Received:
  Name: MasterYi

Timestamp of CHAMPION_KILL event: 399212
Champion killed: Leona (ID: 5)
Victim Damage Received:
  Name: Sylas


In [17]:
# [NOTFINSHED] Print out all champion kill information at specific kill timestamp
def getEventOfChampionKillAtTimestamp(data, targetTimestamp):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL' and event['timestamp'] == targetTimestamp:
                return event

# Example: Get CHAMPION_KILL event data at timestamp 134226
targetTimestamp = 234287
championKillEvent = getEventOfChampionKillAtTimestamp(data, targetTimestamp)

# Print or use the champion_kill_event data as needed
if championKillEvent:
    print(f"CHAMPION_KILL event data:")
    pprint.pprint(championKillEvent)
else:
    print(f"No CHAMPION_KILL event found at timestamp {targetTimestamp}.")


No CHAMPION_KILL event found at timestamp 234287.


In [18]:
def getKillDataForParticipant(data, participantId, participantData):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if killerId == participantId:
                        timestamp = event['timestamp']
                        victimDamageReceived = event.get('victimDamageReceived', [])
                        killPosition = event.get('position', {})  # Added line

                        # Calculate total damage done by the killer to the victim
                        totalDamageDone = sum(
                            damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                            for damageInfo in victimDamageReceived
                            if damageInfo['participantId'] == participantId
                        )

                        # Find names from participantData
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                           f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                           f"Unknown Champion with ID {victimId}")

                        # Find names of assisting champions and their damage
                        assistingChampions = []
                        damageByAssists = []

                        for assistId in assistIds:
                            assistName = next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}")

                            assistDamage = sum(
                                damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                                for damageInfo in victimDamageReceived
                                if damageInfo['participantId'] == assistId
                            )

                            assistingChampions.append(assistName)
                            damageByAssists.append(assistDamage)

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'assistingChampions': assistingChampions,
                            'damageByAssists': damageByAssists,
                            'position': killPosition  # Added line
                        })

    return killData

# Example usage:
participantIdToCheck = 1  # Replace with the desired participant ID
killDataForParticipant = getKillDataForParticipant(data, participantIdToCheck, participantData)

# Display the kill data with total damage done, assisting champions, and damage by assists
for kill in killDataForParticipant:
    assistingChampionsLine = f"Assisting Champions: {', '.join(kill['assistingChampions'])}.\n" if kill['assistingChampions'] else ""
    damageByAssistsLine = f"Damage Done by Assists: {', '.join([f'{damage} by {assist}' for damage, assist in zip(kill['damageByAssists'], kill['assistingChampions'])])}.\n" if kill['assistingChampions'] else ""

    print(f"Timestamp: {kill['timestamp']}\n"
          f"Killer: {kill['killerName']} (ID: {kill['killerId']})\n"
          f"Victim: {kill['victimName']} (ID: {kill['victimId']})\n"
          f"Total Damage Done: {kill['totalDamageDone']}\n"
          f"{assistingChampionsLine}"
          f"{damageByAssistsLine}"
          f"Kill Location: (X: {kill['position'].get('x', 0)}, Y: {kill['position'].get('y', 0)})\n")  # Added line


Timestamp: 272085
Killer: Jax (ID: 1)
Victim: Shen (ID: 6)
Total Damage Done: 440
Assisting Champions: MasterYi.
Damage Done by Assists: 246 by MasterYi.
Kill Location: (X: 3464, Y: 13074)

Timestamp: 283885
Killer: Jax (ID: 1)
Victim: Khazix (ID: 7)
Total Damage Done: 833
Kill Location: (X: 3274, Y: 12199)

Timestamp: 503337
Killer: Jax (ID: 1)
Victim: Shen (ID: 6)
Total Damage Done: 378
Kill Location: (X: 2950, Y: 13188)

Timestamp: 1081605
Killer: Jax (ID: 1)
Victim: Sylas (ID: 8)
Total Damage Done: 721
Assisting Champions: MasterYi, Veigar.
Damage Done by Assists: 1151 by MasterYi, 0 by Veigar.
Kill Location: (X: 4765, Y: 9700)

Timestamp: 1550140
Killer: Jax (ID: 1)
Victim: Khazix (ID: 7)
Total Damage Done: 230
Assisting Champions: Veigar, Varus, Leona.
Damage Done by Assists: 1478 by Veigar, 929 by Varus, 0 by Leona.
Kill Location: (X: 5672, Y: 9180)

Timestamp: 1560344
Killer: Jax (ID: 1)
Victim: Shen (ID: 6)
Total Damage Done: 1276
Assisting Champions: Varus, Leona.
Damage Done

In [19]:
def getAssistsForParticipant(data, targetParticipantId, participantData):
    assistData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if targetParticipantId in assistIds:
                        timestamp = event['timestamp']
                        assistName = next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                          f"Unknown Champion with ID {targetParticipantId}")

                        assistData.append({
                            'timestamp': timestamp,
                            'assistId': targetParticipantId,
                            'assistName': assistName,
                            'position': event.get('position', {'x': 0, 'y': 0})  # Use default coordinates if not present
                        })

    return assistData

# Example usage:
targetParticipantIdToCheck = 1  # Replace with the desired target participant ID
assistDataForParticipant = getAssistsForParticipant(data, targetParticipantIdToCheck, participantData)

# Display the assist data with timestamps and coordinates
for assist in assistDataForParticipant:
    print(f"Timestamp: {assist['timestamp']}\n"
          f"Assist: {assist['assistName']} (ID: {assist['assistId']})\n"
          f"Assist Location: (X: {assist['position'].get('x', 0)}, Y: {assist['position'].get('y', 0)})\n")


Timestamp: 1076595
Assist: Jax (ID: 1)
Assist Location: (X: 4455, Y: 9895)

Timestamp: 1086007
Assist: Jax (ID: 1)
Assist Location: (X: 4053, Y: 9372)

Timestamp: 1158364
Assist: Jax (ID: 1)
Assist Location: (X: 4071, Y: 8585)

Timestamp: 1429372
Assist: Jax (ID: 1)
Assist Location: (X: 4805, Y: 9660)

Timestamp: 1436514
Assist: Jax (ID: 1)
Assist Location: (X: 4190, Y: 9965)

Timestamp: 1557312
Assist: Jax (ID: 1)
Assist Location: (X: 4920, Y: 9304)

Timestamp: 1669150
Assist: Jax (ID: 1)
Assist Location: (X: 9922, Y: 5071)

Timestamp: 1670047
Assist: Jax (ID: 1)
Assist Location: (X: 9973, Y: 4915)

Timestamp: 1670047
Assist: Jax (ID: 1)
Assist Location: (X: 9947, Y: 5009)

Timestamp: 2080390
Assist: Jax (ID: 1)
Assist Location: (X: 13511, Y: 8141)

Timestamp: 2342504
Assist: Jax (ID: 1)
Assist Location: (X: 13072, Y: 12906)



In [20]:
def getTimestampOfChampionKillWithId(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                killerId = event.get('killerId')
                victimId = event.get('victimId')
                victimNames = [victim.get('name') for victim in event.get('victimDamageDealt', [])]
                killerNames = [killer.get('name') for killer in event.get('victimDamageReceived', [])]

                if victimNames:  # Check if there are victim names
                    victimName = victimNames[0]
                    print(f"{victimName} was killed with ID: {victimId}")
                else:
                    print(f"{victimId}")

                if killerNames:  # Check if there are killers
                    # Use the last killer as the one who landed the final hit
                    lastKiller = killerNames[-1]
                    print(f"Killed by {lastKiller} with ID: {killerId}")
                    print(f"Timestamp of CHAMPION_KILL: {timestamp}\n")


#getTimestampOfChampionKillWithId(data)


In [21]:
def displayParticipantParameters(data, participantIds):

    # Display the extracted parameter values
    for participantId in participantIds:
        print(f"Participant {participantId} Parameters:")
        for idx, frameData in enumerate(participantFrames):
            damageStats = frameData.get(participantId, {}).get('damageStats', {})
            totalDamageDone = damageStats.get('totalDamageDone', 0)
            totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
            totalDamageTaken = damageStats.get('totalDamageTaken', 0)

            print(f"Frame {idx + 1}:")
            print(f"  Total Damage Done: {totalDamageDone}")
            print(f"  Total Damage Done To Champions: {totalDamageDoneToChampions}")
            print(f"  Total Damage Taken: {totalDamageTaken}")

# Example usage:
# Assuming 'data' is the loaded JSON data
displayParticipantParameters(data, ['2'])


Participant 2 Parameters:
Frame 1:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 2:
  Total Damage Done: 145
  Total Damage Done To Champions: 145
  Total Damage Taken: 0
Frame 3:
  Total Damage Done: 3255
  Total Damage Done To Champions: 145
  Total Damage Taken: 755
Frame 4:
  Total Damage Done: 10871
  Total Damage Done To Champions: 145
  Total Damage Taken: 1725
Frame 5:
  Total Damage Done: 16416
  Total Damage Done To Champions: 145
  Total Damage Taken: 2765
Frame 6:
  Total Damage Done: 17705
  Total Damage Done To Champions: 504
  Total Damage Taken: 3747
Frame 7:
  Total Damage Done: 27259
  Total Damage Done To Champions: 504
  Total Damage Taken: 4768
Frame 8:
  Total Damage Done: 28691
  Total Damage Done To Champions: 1502
  Total Damage Taken: 5764
Frame 9:
  Total Damage Done: 32242
  Total Damage Done To Champions: 1502
  Total Damage Taken: 6129
Frame 10:
  Total Damage Done: 37783
  Total Damage Done To Champions: 2550
  T

In [22]:
def getParticipantIdByKill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events',[]):
            eventType = event.get('type')
            if eventType == 'CHAMPION_KILL':
                killerId = event.get('killerId')
                print(f"Participant id: {killerId}") 

#getParticipantIdByKill(data)

In [23]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions

    # Print the total damage done to champions only for the specified frames
    print(f"PRE")
    print(f"Frame {startFrame}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"POST")
    print(f"Frame {endFrame}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")

    # Calculate and print the difference
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    print(f"\nTotal Damage Done To Champions Difference: {difference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '2', 7, 8)


PRE
Frame 7:
  Total Damage Done To Champions: 504
POST
Frame 8:
  Total Damage Done To Champions: 1502

Total Damage Done To Champions Difference: 998


In [24]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0
    totalDamageToChampionsExtraStart = 0
    totalDamageToChampionsExtraEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
            totalDamageToChampionsExtraStart = totalDamageDoneToChampionsExtra
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions
            totalDamageToChampionsExtraEnd = totalDamageDoneToChampionsExtra

    # Print the total damage done to champions for the participant and extra participant
    print(f"PRE")
    print(f"Frame {startFrame} - Participant {participantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"Frame {startFrame} - Extra Participant {extraParticipantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraStart}")
    
    print(f"POST")
    print(f"Frame {endFrame} - Participant {participantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")
    print(f"Frame {endFrame} - Extra Participant {extraParticipantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraEnd}")

    # Calculate and print the differences
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    extraDifference = totalDamageToChampionsExtraEnd - totalDamageToChampionsExtraStart
    print(f"\nTotal Damage Done To Champions Difference (Participant {participantId}): {difference}")
    print(f"Total Damage Done To Champions Difference (Extra Participant {extraParticipantId}): {extraDifference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '5', 4, 10)

PRE
Frame 4 - Participant 5:
  Total Damage Done To Champions: 0
Frame 4 - Extra Participant 0:
  Total Damage Done To Champions: 0
POST
Frame 10 - Participant 5:
  Total Damage Done To Champions: 1939
Frame 10 - Extra Participant 0:
  Total Damage Done To Champions: 0

Total Damage Done To Champions Difference (Participant 5): 1939
Total Damage Done To Champions Difference (Extra Participant 0): 0


In [25]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Initialize variables to store total damage done parameters
    totalDamageDoneStart = 0
    totalDamageDoneEnd = 0
    totalDamageDoneExtraStart = 0
    totalDamageDoneExtraEnd = 0

    totalDamageDoneToChampionsStart = 0
    totalDamageDoneToChampionsEnd = 0
    totalDamageDoneToChampionsExtraStart = 0
    totalDamageDoneToChampionsExtraEnd = 0

    totalDamageTakenStart = 0
    totalDamageTakenEnd = 0
    totalDamageTakenExtraStart = 0
    totalDamageTakenExtraEnd = 0
    
    totalDamageTakenCompared = 0
    totalDamageDoneCompared = 0
    totalDamageDoneToChampionsCompared = 0

    # Display the extracted parameter values and calculate total damage done parameters
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDone = damageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTaken = damageStats.get('totalDamageTaken', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneExtra = extraDamageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTakenExtra = extraDamageStats.get('totalDamageTaken', 0)

        if idx + 1 == startFrame:
            totalDamageDoneStart = totalDamageDone
            totalDamageDoneToChampionsStart = totalDamageDoneToChampions
            totalDamageTakenStart = totalDamageTaken
            totalDamageDoneExtraStart = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraStart = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraStart = totalDamageTakenExtra
        elif idx + 1 <= endFrame:
            totalDamageDoneEnd = totalDamageDone
            totalDamageDoneToChampionsEnd = totalDamageDoneToChampions
            totalDamageTakenEnd = totalDamageTaken
            totalDamageDoneExtraEnd = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraEnd = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraEnd = totalDamageTakenExtra
    
    # Print the pre and post data for the first participant
    print(f"Participant {participantId}:")
    print(f"  Frame {startFrame} - Total Damage Done: {totalDamageDoneStart}")
    print(f"  Frame {endFrame} - Total Damage Done: {totalDamageDoneEnd}")
    print(f"  Frame {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsStart}")
    print(f"  Frame {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsEnd}")
    print(f"  Frame {startFrame} - Total Damage Taken: {totalDamageTakenStart}")
    print(f"  Frame {endFrame} - Total Damage Taken: {totalDamageTakenEnd}")

    # Print the data for the extra participant
    print(f"\nEnemy Laner:")
    print(f"  Frame {startFrame} - Total Damage Done: {totalDamageDoneExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Done: {totalDamageDoneExtraEnd}")
    print(f"  Frame {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraEnd}")
    print(f"  Frame {startFrame} - Total Damage Taken: {totalDamageTakenExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Taken: {totalDamageTakenExtraEnd}")

    # Calculate and print the differences
    damageDoneDifference = totalDamageDoneEnd - totalDamageDoneStart
    extraDamageDoneDifference = totalDamageDoneExtraEnd - totalDamageDoneExtraStart
    damageDoneToChampionsDifference = totalDamageDoneToChampionsEnd - totalDamageDoneToChampionsStart
    extraDamageDoneToChampionsDifference = totalDamageDoneToChampionsExtraEnd - totalDamageDoneToChampionsExtraStart
    damageTakenDifference = totalDamageTakenEnd - totalDamageTakenStart
    extraDamageTakenDifference = totalDamageTakenExtraEnd - totalDamageTakenExtraStart

    print(f"\nTotal Damage Done Difference (Participant {participantId}): {damageDoneDifference}")
    print(f"Total Damage Done Difference (Enemy Laner): {extraDamageDoneDifference}")
    print(f"Total Damage Done To Champions Difference (Participant {participantId}): {damageDoneToChampionsDifference}")
    print(f"Total Damage Done To Champions Difference (Enemy Laner): {extraDamageDoneToChampionsDifference}")
    print(f"Total Damage Taken Difference (Participant {participantId}): {damageTakenDifference}")
    print(f"Total Damage Taken Difference (Enemy Laner): {extraDamageTakenDifference}")
    
    # Calculate differences between participant and enemy
    totalDamageTakenCompared = damageTakenDifference - extraDamageTakenDifference
    totalDamageDoneCompared = damageDoneDifference - extraDamageDoneDifference
    totalDamageDoneToChampionsCompared = damageDoneToChampionsDifference - extraDamageDoneToChampionsDifference
    
    if totalDamageTakenCompared >= 0:
        print(f"\nParticipant {participantId} took {totalDamageTakenCompared} more damage than enemy laner in given timeframe")
    else:
        print(f"\nParticipant {participantId} took {abs(totalDamageTakenCompared)} less damage than enemy laner in given timeframe")

    if totalDamageDoneToChampionsCompared >= 0:
        print(f"\nParticipant {participantId} beat enemy laner in given timeframe with total damage done difference of: {totalDamageDoneToChampionsCompared}")
    else:
        print(f"\nParticipant {participantId} was beat by enemy laner in given timeframe with total damage done difference of: {abs(totalDamageDoneToChampionsCompared)}")

    if totalDamageDoneCompared >= 0:
        print(f"\nParticipant {participantId} beat enemy laner in given timeframe with total damage done difference of: {totalDamageDoneCompared}")
    else:
        print(f"\nParticipant {participantId} was beat by enemy laner in given timeframe with total damage done difference of: {abs(totalDamageDoneCompared)}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '2', 4, 6)


Participant 2:
  Frame 4 - Total Damage Done: 10871
  Frame 6 - Total Damage Done: 17705
  Frame 4 - Total Damage Done To Champions: 145
  Frame 6 - Total Damage Done To Champions: 504
  Frame 4 - Total Damage Taken: 1725
  Frame 6 - Total Damage Taken: 3747

Enemy Laner:
  Frame 4 - Total Damage Done: 11658
  Frame 6 - Total Damage Done: 18756
  Frame 4 - Total Damage Done To Champions: 0
  Frame 6 - Total Damage Done To Champions: 1483
  Frame 4 - Total Damage Taken: 2276
  Frame 6 - Total Damage Taken: 4269

Total Damage Done Difference (Participant 2): 6834
Total Damage Done Difference (Enemy Laner): 7098
Total Damage Done To Champions Difference (Participant 2): 359
Total Damage Done To Champions Difference (Enemy Laner): 1483
Total Damage Taken Difference (Participant 2): 2022
Total Damage Taken Difference (Enemy Laner): 1993

Participant 2 took 29 more damage than enemy laner in given timeframe

Participant 2 was beat by enemy laner in given timeframe with total damage done diff

In [26]:
#postGameData

In [27]:
postChallengeData

[{'12AssistStreakCount': 0,
  'abilityUses': 241,
  'acesBefore15Minutes': 0,
  'alliedJungleMonsterKills': 2,
  'baronTakedowns': 1,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 375,
  'buffsStolen': 0,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 740.6338404924327,
  'damageTakenOnTeamPercentage': 0.2562878615249701,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 9,
  'dodgeSkillShotsSmallWindow': 1,
  'doubleAces': 0,
  'dragonTakedowns': 0,
  'earliestBaron': 1845.9990139,
  'earliestElderDragon': 2305.4527350999997,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 0,
  'elderDragonKillsWithOpposingSoul': 1,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 24,
  'enemyJungleMonsterKills': 3,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMonsterKillsWithin30SecondsOfSpawn': 0,
  'epicMonsterSteals': 0,
  'epicMonsterStolenWithoutSmite': 0,
  'firstTurretKilled': 1,
  'firstTurretKilledTime

In [28]:
def getVisionScoreAdvantageLaneOpponent(postData):
    
    for participant in postData['info']['participants']:
        challengesData = participant.get('challenges', {})
        visionScoreAdvantageLaneOpponent = challengesData.get('visionScoreAdvantageLaneOpponent', None)

        if visionScoreAdvantageLaneOpponent is not None:
            print(f"Participant ID: {participant['participantId']}, "
                  f"Vision Score Advantage Lane Opponent: {visionScoreAdvantageLaneOpponent}")
        else:
            print(f"Participant ID: {participant['participantId']}, "
                  "No Vision Score Advantage Lane Opponent data.")
getVisionScoreAdvantageLaneOpponent(postData)

Participant ID: 1, Vision Score Advantage Lane Opponent: -0.3503682613372803
Participant ID: 2, Vision Score Advantage Lane Opponent: 0.44010066986083984
Participant ID: 3, Vision Score Advantage Lane Opponent: -0.18380868434906006
Participant ID: 4, Vision Score Advantage Lane Opponent: 0.6289721727371216
Participant ID: 5, Vision Score Advantage Lane Opponent: -0.5002038478851318
Participant ID: 6, Vision Score Advantage Lane Opponent: 0.5393335819244385
Participant ID: 7, Vision Score Advantage Lane Opponent: -0.30560410022735596
Participant ID: 8, Vision Score Advantage Lane Opponent: 0.2252030372619629
Participant ID: 9, Vision Score Advantage Lane Opponent: -0.3861159682273865
Participant ID: 10, Vision Score Advantage Lane Opponent: 1.0008158683776855


In [29]:
def getControlWardTimeCoverageInRiverOrEnemyHalf(postData):
    for participant in postData['info']['participants']:
        challengesData = participant.get('challenges',{})
        controlWardTimeCoverageInRiverOrEnemyHalf = challengesData.get('controlWardTimeCoverageInRiverOrEnemyHalf', None)
        
        if controlWardTimeCoverageInRiverOrEnemyHalf is not None:
            print(f"Participant ID: {participant['participantId']}, "
                 f"Control Ward Time Coverage In River or Enemy Half: {controlWardTimeCoverageInRiverOrEnemyHalf}")
        else:
            print(f"Participant ID: {participant['participantId']}, "
                  "No Data")
            
getControlWardTimeCoverageInRiverOrEnemyHalf(postData)

Participant ID: 1, No Data
Participant ID: 2, Control Ward Time Coverage In River or Enemy Half: 0.44759596864850204
Participant ID: 3, No Data
Participant ID: 4, No Data
Participant ID: 5, No Data
Participant ID: 6, Control Ward Time Coverage In River or Enemy Half: 0.8611719733305505
Participant ID: 7, No Data
Participant ID: 8, No Data
Participant ID: 9, No Data
Participant ID: 10, No Data


In [30]:
def getTakedowns(postData):
    for participant in postData['info']['participants']:
        challengesData = participant.get('challenges',{})
        takedowns = challengesData.get('takedowns', None)
        
        if takedowns is not None:
            print(f"Participant ID: {participant['participantId']}, "
                 f"data: {takedowns}")
        else:
            print(f"Participant ID: {participant['participantId']}, "
                  "No Data")
            
getTakedowns(postData)

Participant ID: 1, data: 21
Participant ID: 2, data: 34
Participant ID: 3, data: 27
Participant ID: 4, data: 27
Participant ID: 5, data: 28
Participant ID: 6, data: 21
Participant ID: 7, data: 24
Participant ID: 8, data: 15
Participant ID: 9, data: 14
Participant ID: 10, data: 12


In [31]:
def blueCase3(x, y):
    if x < 11.4:
        if x<1.73:
            print("champion is safe")
            return True
        elif y<1.8:
            print("champion is safe")
            return True
        elif x < 5.2 and y < 5.2:
            print("champion is safe")
            return True

    print("champion is in danger")
    return False

# Test with some coordinates
coordinate_set1 = (2, 6)
coordinate_set2 = (4, 2)
coordinate_set3 = (6, 1)
coordinate_set4 = (2, 2)
coordinate_set5 = (1.5, 1.5)
coordinate_set6 = (1, 4)
coordinate_set7 = (1, 6)

blueCase3(*coordinate_set1)  # should print "champion is in danger"
blueCase3(*coordinate_set2)  # should print "champion is safe"
blueCase3(*coordinate_set3)  # should print "champion is safe"
blueCase3(*coordinate_set4)  # should print "champion is safe"
blueCase3(*coordinate_set5)  # should print "champion is safe"
blueCase3(*coordinate_set6)  # should print "champion is safe"
blueCase3(*coordinate_set7)  # should print "champion is safe"


champion is in danger
champion is safe
champion is safe
champion is safe
champion is safe
champion is safe
champion is safe


True

In [32]:
def redCase3(x, y):
    if y > 3:
        if x>13.1:
            print("champion is safe")
            return True
        elif y>13.2:
            print("champion is safe")
            return True
        elif x > 9.8 and y > 9.8:
            print("champion is safe")
            return True

    print("champion is in danger")
    return False

# Test with some coordinates
coordinate_set1 = (14, 14)
coordinate_set2 = (13, 14)
coordinate_set3 = (14, 13)
coordinate_set4 = (5, 14)
coordinate_set5 = (14, 5)
coordinate_set6 = (11, 11)
coordinate_set7 = (14, 2)
coordinate_set8 = (8, 8)


redCase3(*coordinate_set1)  # should print "champion is safe"
redCase3(*coordinate_set2)  # should print "champion is safe"
redCase3(*coordinate_set3)  # should print "champion is safe"
redCase3(*coordinate_set4)  # should print "champion is safe"
redCase3(*coordinate_set5)  # should print "champion is safe"
redCase3(*coordinate_set6)  # should print "champion is safe"
redCase3(*coordinate_set7)  # should print "champion is in danger"
redCase3(*coordinate_set8)  # should print "champion is in danger"

champion is safe
champion is safe
champion is safe
champion is safe
champion is safe
champion is safe
champion is in danger
champion is in danger


False

Relevant data from postGame data:

- visionScoreAdvantageLaneOpponent
- visionScore
- killParticipation
- goldPerMinute
- totalDamageDealtToChampions
- totalDamageTaken
- controlWardTimeCoverageInRiverOrEnemyHalf
- controlWardsPlaced
- soloKills
- stealthWardsPlaced
- wardTakedowns
- teamBaronKills
- teamRiftHeraldKills
- teamElderDragonKills
- dragonKills
- deathsByEnemyChamps
- dragonTakedowns
- kda
- Takedowns
- turretTakedowns
- laneMinionsFirst10Minutes (Maybe?)

In [2]:
import numpy as np
import pandas as pd

csv_file_path = 'E:\DistanceTesting\match_data.csv'

df = pd.read_csv(csv_file_path)

data = df[['matchId', 'participantId3', 'participantId8']]
y_data = data.to_numpy()

print(y_data)

only participants
[[0.8  0.7 ]
 [0.2  0.7 ]
 [0.7  0.8 ]
 [0.7  0.75]
 [0.65 0.4 ]
 [0.7  0.75]
 [0.4  0.8 ]
 [0.7  0.8 ]
 [0.6  0.9 ]
 [0.6  0.9 ]
 [0.6  0.85]
 [0.8  0.8 ]
 [0.7  0.8 ]
 [0.9  0.6 ]
 [0.55 0.7 ]
 [0.65 0.65]
 [0.55 0.65]
 [0.3  0.8 ]
 [0.55 0.7 ]
 [0.75 0.3 ]
 [0.65 0.5 ]
 [0.65 0.45]
 [0.55 0.7 ]
 [0.4  0.7 ]
 [0.7  0.7 ]
 [0.5  0.85]
 [0.5  0.7 ]
 [0.78 0.7 ]
 [0.75 0.6 ]
 [0.5  0.65]
 [0.85 0.2 ]
 [0.5  0.65]
 [0.65 0.65]
 [0.7  0.65]
 [0.6  0.65]
 [0.65 0.65]
 [0.4  0.75]
 [0.6  0.6 ]
 [0.65 0.7 ]
 [0.7  0.4 ]]
________________
everything
[['EUW1_6851860388' 0.8 0.7]
 ['EUW1_6851835177' 0.2 0.7]
 ['EUW1_6854080044' 0.7 0.8]
 ['EUW1_6854039048' 0.7 0.75]
 ['EUW1_6844839591' 0.65 0.4]
 ['EUW1_6846029145' 0.7 0.75]
 ['EUW1_6848297504' 0.4 0.8]
 ['EUW1_6848033436' 0.7 0.8]
 ['EUW1_6849549607' 0.6 0.9]
 ['EUW1_6849483028' 0.6 0.9]
 ['EUW1_6846590564' 0.6 0.85]
 ['EUW1_6846961917' 0.8 0.8]
 ['EUW1_6847078890' 0.7 0.8]
 ['EUW1_6847115159' 0.9 0.6]
 ['EUW1_6847177914' 0.5

In [5]:
import requests
import pandas as pd
import os

def fetch_and_save_match_data(match_id, save_path):
    """
    Fetch match data using the match ID and save the response body to a file.

    :param match_id: The ID of the match to fetch.
    :param save_path: The directory where the match data files will be saved.
    """
    # Define the API URL
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"

    # Define the request headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-21fc8103-1d30-44ac-b7d1-1ea632c09bed"  # Replace with your actual API key
    }

    # Make the API call
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Ensure the save directory exists
        os.makedirs(save_path, exist_ok=True)

        # Define the path for the output file
        file_path = os.path.join(save_path, f"{match_id}.json")

        # Save the response body to a file
        # Save the response body to a file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(response.text)

        print(f"Match data for {match_id} saved to {file_path}")
    else:
        print(f"Failed to fetch match data for {match_id}. Status code: {response.status_code}")

def read_match_ids_and_fetch_data(csv_file_path, save_path):
    # Read the CSV file to get the match IDs
    df = pd.read_csv(csv_file_path)

    # Loop through each row and fetch data for each match ID
    for match_id in df['matchId']:
        fetch_and_save_match_data(match_id, save_path)

# Example usage
csv_file_path = "E:\\DistanceTesting\\match_data.csv"  # The path to your CSV file
save_path = "E:\\DistanceTesting\\samplematches\\postMatch"  # The directory to save the match data files
read_match_ids_and_fetch_data(csv_file_path, save_path)


Match data for EUW1_6851860388 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6851860388.json
Match data for EUW1_6851835177 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6851835177.json
Match data for EUW1_6854080044 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6854080044.json
Match data for EUW1_6854039048 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6854039048.json
Match data for EUW1_6844839591 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6844839591.json
Match data for EUW1_6846029145 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6846029145.json
Match data for EUW1_6848297504 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6848297504.json
Match data for EUW1_6848033436 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6848033436.json
Match data for EUW1_6849549607 saved to E:\DistanceTesting\samplematches\postMatch\EUW1_6849549607.json
Match data for EUW1_6849483028 saved to E:\DistanceTesting\sampl

In [7]:
import requests
import pandas as pd
import os

def fetch_and_save_match_timeline(match_id, save_path):
    """
    Fetch match timeline data using the match ID and save the response body to a file.

    :param match_id: The ID of the match to fetch the timeline for.
    :param save_path: The directory where the match timeline data files will be saved.
    """
    # Define the API URL with the `/timeline` suffix
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"

    # Define the request headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-21fc8103-1d30-44ac-b7d1-1ea632c09bed"  # Replace with your actual API key
    }

    # Make the API call
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Ensure the save directory exists
        os.makedirs(save_path, exist_ok=True)

        # Define the path for the output file
        file_path = os.path.join(save_path, f"{match_id}.json")

        # Save the response body to a file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(response.text)
        print(f"Match timeline data for {match_id} saved to {file_path}")
    else:
        print(f"Failed to fetch match timeline data for {match_id}. Status code: {response.status_code}")

def read_match_ids_and_fetch_timeline(csv_file_path, save_path):
    # Read the CSV file to get the match IDs
    df = pd.read_csv(csv_file_path)

    # Loop through each row and fetch timeline data for each match ID
    for match_id in df['matchId']:
        fetch_and_save_match_timeline(match_id, save_path)

# Example usage
csv_file_path = "E:\\DistanceTesting\\match_data.csv"  # The path to your CSV file
save_path_timeline = "E:\\DistanceTesting\\samplematches\\timeline"  # The directory to save the timeline data files
read_match_ids_and_fetch_timeline(csv_file_path, save_path_timeline)


Match timeline data for EUW1_6851860388 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6851860388.json
Match timeline data for EUW1_6851835177 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6851835177.json
Match timeline data for EUW1_6854080044 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6854080044.json
Match timeline data for EUW1_6854039048 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6854039048.json
Match timeline data for EUW1_6844839591 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6844839591.json
Match timeline data for EUW1_6846029145 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6846029145.json
Match timeline data for EUW1_6848297504 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6848297504.json
Match timeline data for EUW1_6848033436 saved to E:\DistanceTesting\samplematches\timeline\EUW1_6848033436.json
Match timeline data for EUW1_6849549607 saved to E:\DistanceTesting\samplematches\timeline\EUW1_68495496